This script preprocesses and analyzes a dataset of air quality measurements from Barcelona over the last five years. It loads the data, selects relevant columns, ensures the data types are correct, and computes daily mean concentrations for selected pollutants. The script then identifies missing data intervals, checks station data consistency, and applies ANOVA to examine differences between traffic and background station types. Finally, it explores multiple imputation techniques to handle missing values effectively.


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

In [3]:

file_path = "df_Barcelona_pivoted_5_years.csv"
df_barcelona = pd.read_csv(file_path, low_memory=False)
print(df_barcelona.shape)
df_barcelona.head()

(346412, 15)


,CODI EOI,NOM ESTACIO,CO,H2S,HCNM,HCT,NO,NO2,NOX,O3,PM10,PM2.5,PS,SO2,Date_time
0,8019004,Barcelona (Poblenou),NaN,NaN,NaN,NaN,3.0,27.0,31.0,NaN,18.0,NaN,NaN,NaN,2020-01-01 01:00:00
1,8019004,Barcelona (Poblenou),NaN,NaN,NaN,NaN,3.0,31.0,36.0,NaN,13.0,NaN,NaN,NaN,2020-01-01 02:00:00
2,8019004,Barcelona (Poblenou),NaN,NaN,NaN,NaN,2.0,27.0,31.0,NaN,10.0,NaN,NaN,NaN,2020-01-01 03:00:00
3,8019004,Barcelona (Poblenou),NaN,NaN,NaN,NaN,1.0,15.0,17.0,NaN,9.0,NaN,NaN,NaN,2020-01-01 04:00:00
4,8019004,Barcelona (Poblenou),NaN,NaN,NaN,NaN,2.0,22.0,25.0,NaN,8.0,NaN,NaN,NaN,2020-01-01 05:00:00


In [7]:
df_barcelona['Date_time'] = pd.to_datetime(df_barcelona['Date_time'])

In [19]:
df_barcelona['Date_time'].min()
df_barcelona['Date_time'].max()

Timestamp('2024-03-07 04:00:00')

,Date_time,NOM ESTACIO,CODI EOI,CO,H2S,HCNM,HCT,NO,NO2,NOX,O3,PM10,PM2.5,PS,SO2
0,2019-03-07 04:00:00,Barcelona (Poblenou),8019004.0,NaN,NaN,NaN,NaN,1.0,15.0,17.0,NaN,2.0,NaN,NaN,NaN
1,2019-03-07 04:00:00,Barcelona (Sants),8019042.0,NaN,NaN,NaN,NaN,1.0,23.0,25.0,NaN,NaN,NaN,NaN,NaN
2,2019-03-07 04:00:00,Barcelona (Eixample),8019043.0,0.2,NaN,NaN,NaN,4.0,29.0,34.0,46.0,5.0,NaN,NaN,1.0
3,2019-03-07 04:00:00,Barcelona (Gràcia - Sant Gervasi),8019044.0,0.2,NaN,NaN,NaN,1.0,12.0,12.0,66.0,2.0,NaN,NaN,1.0
4,2019-03-07 04:00:00,Barcelona (Ciutadella),8019050.0,NaN,NaN,NaN,NaN,1.0,21.0,21.0,56.0,NaN,NaN,NaN,NaN


In [22]:
df_barcelona.shape

(346412, 8)

In [8]:

# Seleccionar solo las columnas deseadas junto con 'Date_time' y 'NOM ESTACIO'
selected_columns = ['NOM ESTACIO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'Date_time']
filtered_data = df_barcelona[selected_columns]

# Asegurarse de que todas las columnas de contaminantes sean de tipo flotante
contaminant_columns = ['NO', 'NO2', 'NOX', 'O3', 'PM10']
filtered_data[contaminant_columns] = filtered_data[contaminant_columns].astype(float)

# Agrupar por día y estación, y calcular la media de los contaminantes seleccionados
daily_means_filtered = filtered_data.groupby([pd.Grouper(key='Date_time', freq='D'), 'NOM ESTACIO']).mean()

# Resetear el índice para que 'Date_time' y 'NOM ESTACIO' sean columnas de nuevo
daily_means_filtered.reset_index(inplace=True)



<ipython-input-8-7070656a3edc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[contaminant_columns] = filtered_data[contaminant_columns].astype(float)


In [9]:
daily_means_filtered.head()

,Date_time,NOM ESTACIO,NO,NO2,NOX,O3,PM10
0,2019-03-07,Barcelona (Ciutadella),4.684211,30.210526,37.052632,53.263158,NaN
1,2019-03-07,Barcelona (Eixample),12.850000,35.100000,54.600000,45.100000,18.900000
2,2019-03-07,Barcelona (Gràcia - Sant Gervasi),9.900000,37.300000,52.050000,47.200000,17.400000
3,2019-03-07,Barcelona (Observatori Fabra),12.166667,12.250000,30.416667,82.833333,8.714286
4,2019-03-07,Barcelona (Palau Reial),5.500000,20.850000,29.450000,51.400000,11.800000


In [31]:
daily_means_filtered.shape

(14624, 7)

In [10]:
daily_means_filtered['PM10'].max()

160.83333333333334

: 

In [30]:
# Generate complete date range
date_range = pd.date_range(daily_means_filtered['Date_time'].min(), daily_means_filtered['Date_time'].max(), freq='D')

# Get unique stations
stations = daily_means_filtered['NOM ESTACIO'].unique()

# Create multi-index with all combinations of dates and stations
multi_index = pd.MultiIndex.from_product([date_range, stations], names=['Date_time', 'NOM ESTACIO'])

# Reindex df_barcelona with the generated multi-index
daily_means_filtered = daily_means_filtered.set_index(['Date_time', 'NOM ESTACIO']).reindex(multi_index)

# Reset the index to separate Date_time and NOM ESTACIO columns
daily_means_filtered = daily_means_filtered.reset_index()

# Fill missing values with NaN
daily_means_filtered = daily_means_filtered.fillna(np.nan)

In [32]:
daily_means_filtered.isna().sum()

Date_time         0
NOM ESTACIO       0
NO              232
NO2             235
NOX             459
O3             3777
PM10           4057
dtype: int64

In [33]:
daily_means_filtered.shape

(14624, 7)

In [9]:
import pandas as pd

def check_nan_intervals(station, pollutant):
    daily_means_station = daily_means_filtered[daily_means_filtered['NOM ESTACIO'] == station]
    daily_means_station['Date_time'] = pd.to_datetime(daily_means_station['Date_time'])
    daily_means_station['Is_NaN'] = daily_means_station[pollutant].isna()
    daily_means_station['shifted'] = daily_means_station['Is_NaN'].shift(1, fill_value=daily_means_station['Is_NaN'].iloc[0])
    changes = daily_means_station[daily_means_station['Is_NaN'] != daily_means_station['shifted']]
    start_times = changes[(changes['Is_NaN'] == True) & (changes['shifted'] == False)]['Date_time']
    end_times = changes[(changes['Is_NaN'] == False) & (changes['shifted'] == True)]['Date_time']
    
    if len(start_times) > len(end_times):
        end_times = pd.concat([end_times,pd.Series([daily_means_station['Date_time'].max()] * (len(start_times) - len(end_times)))], ignore_index=True)
        

    intervals = pd.DataFrame({'Start': start_times.values, 'End': end_times.values, 'Station': [station] * len(start_times.values), 'Pollutant': [pollutant] * len(start_times.values)})

    return intervals

intervals = []

for station in daily_means_filtered['NOM ESTACIO'].unique():
    for pollutant in contaminant_columns:
        intervals.append(check_nan_intervals(station, pollutant))

na_intervals = pd.concat(intervals, ignore_index=True)


<ipython-input-9-45e1455a4f0e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_means_station['Date_time'] = pd.to_datetime(daily_means_station['Date_time'])
<ipython-input-9-45e1455a4f0e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_means_station['Is_NaN'] = daily_means_station[pollutant].isna()
<ipython-input-9-45e1455a4f0e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [10]:
daily_means_filtered['NOM ESTACIO'].unique()

array(['Barcelona (Ciutadella)', 'Barcelona (Eixample)',
       'Barcelona (Gràcia - Sant Gervasi)',
       'Barcelona (Observatori Fabra)', 'Barcelona (Palau Reial)',
       'Barcelona (Parc Vall Hebron)', 'Barcelona (Poblenou)',
       'Barcelona (Sants)'], dtype=object)

In [34]:
na_intervals.to_csv('na_intervals.csv', index=False)


ANOVA:   DIFERENCIA ENTRE ESTACIONES TRAFFIC Y BACKGROUND 

In [9]:
# BEGIN: Add column 'type' to daily_means_filtered DataFrame
daily_means_filtered['type'] = 'background'
daily_means_filtered.loc[daily_means_filtered['NOM ESTACIO'].isin(['Barcelona (Gràcia - Sant Gervasi)', 'Barcelona (Eixample)']), 'type'] = 'traffic'
# END: Add column 'type' to daily_means_filtered DataFrame

daily_means_filtered.tail()

,Date_time,NOM ESTACIO,NO,NO2,NOX,O3,PM10,type
14556,2024-03-07,Barcelona (Observatori Fabra),1.0,5.0,5.2,90.2,9.6,background
14557,2024-03-07,Barcelona (Palau Reial),1.0,15.2,16.2,49.8,5.6,background
14558,2024-03-07,Barcelona (Parc Vall Hebron),11.0,48.2,64.8,22.0,14.4,background
14559,2024-03-07,Barcelona (Poblenou),3.2,51.8,56.2,NaN,20.2,background
14560,2024-03-07,Barcelona (Sants),1.0,22.8,23.2,NaN,NaN,background


In [13]:
# Ajustar el modelo ANOVA
model = ols('NO ~ C(type)', data=df_barcelona).fit()

# Ver el ANOVA table
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA DataFrame
print(anova_table)


                sum_sq        df            F  PR(>F)
C(type)   6.199941e+06       1.0  18196.70392     0.0
Residual  1.158004e+08  339872.0          NaN     NaN


In [36]:
daily_means_filtered.rename(columns={'NOM ESTACIO': 'station'}, inplace=True)


In [30]:
for x in ['type','station']:
    for pollutant in contaminant_columns:
        # Ajustar el modelo ANOVA
        model = ols(f'{pollutant} ~ C({x})', data=daily_means_filtered).fit()
        # Ver el ANOVA table
        anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA DataFrame
        print(f'\nANOVA for {x} and {pollutant}')
        print(anova_table)


ANOVA for type and NO
                sum_sq       df            F  PR(>F)
C(type)   2.654371e+05      1.0  1929.326771     0.0
Residual  1.979779e+06  14390.0          NaN     NaN

ANOVA for type and NO2
                sum_sq       df            F  PR(>F)
C(type)   5.730145e+05      1.0  3379.970506     0.0
Residual  2.439063e+06  14387.0          NaN     NaN

ANOVA for type and NOX
                sum_sq       df            F  PR(>F)
C(type)   2.282001e+06      1.0  2824.035859     0.0
Residual  1.144461e+07  14163.0          NaN     NaN

ANOVA for type and O3
                sum_sq       df           F         PR(>F)
C(type)   5.457160e+05      1.0  1285.58026  3.595794e-266
Residual  4.603594e+06  10845.0         NaN            NaN

ANOVA for type and PM10
                sum_sq       df           F         PR(>F)
C(type)   7.225728e+04      1.0  590.237947  6.473889e-127
Residual  1.293374e+06  10565.0         NaN            NaN

ANOVA for station and NO
                  sum_sq

c:\Users\ninab\anaconda3\lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 7, but rank is 5
  warnings.warn('covariance of constraints does not have full '
c:\Users\ninab\anaconda3\lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 7, but rank is 1
  warnings.warn('covariance of constraints does not have full '


MISSING VALUES IMPUTATION : Multiple imputation by chained equation (MICE)

In [ ]:
import pandas as pd
from statsmodels.imputation import mice

# Suponiendo que 'data' es tu DataFrame
data = pd.read_csv('tu_archivo.csv')

# Inicializar el modelo MICE
imp = mice.MICEData(data)

# Ejecutar múltiples imputaciones. Por ejemplo, 10 imputaciones.
mice_results = mice.MICE('formula', 'PMM', imp).fit(10, skip_burnin=True)

# Imputar los datos y retornar el DataFrame completo imputado
imputed_data = imp.data

In [37]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor  # Puedes usar otro estimador
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Preparar transformadores para columnas numéricas y categóricas
numeric_features = ['NO', 'NO2', 'NOX', 'O3', 'PM10']
numeric_transformer = IterativeImputer(RandomForestRegressor())

categorical_features = ['station']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Crear un transformador de columnas para aplicar transformaciones adecuadas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [38]:
# Fit the OneHotEncoder
categorical_transformer.fit(daily_means_filtered[categorical_features])

# Get the one-hot encoded station column names
station_columns = categorical_transformer.get_feature_names_out(input_features=categorical_features)

station_columns

array(['station_Barcelona (Ciutadella)', 'station_Barcelona (Eixample)',
       'station_Barcelona (Gràcia - Sant Gervasi)',
       'station_Barcelona (Observatori Fabra)',
       'station_Barcelona (Palau Reial)',
       'station_Barcelona (Parc Vall Hebron)',
       'station_Barcelona (Poblenou)', 'station_Barcelona (Sants)'],
      dtype=object)

In [39]:
# Crear un pipeline con el preprocesador y un estimador final para la imputación
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Ajustar e imputar los datos en un solo paso
imputed_data = pipeline.fit_transform(daily_means_filtered)


# Convertir los resultados imputados de nuevo a DataFrame para fácil manipulación
imputed_data = pd.DataFrame(imputed_data, columns=numeric_features+ ['station_Barcelona (Ciutadella)', 'station_Barcelona (Eixample)','station_Barcelona (Gràcia - Sant Gervasi)','station_Barcelona (Observatori Fabra)','station_Barcelona (Palau Reial)','station_Barcelona (Parc Vall Hebron)','station_Barcelona (Poblenou)', 'station_Barcelona (Sants)'])
print(imputed_data.shape)

imputed_data.insert(0, 'Date_time', daily_means_filtered['Date_time'].values)
imputed_data.head()


(14624, 13)


c:\Users\ninab\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:713: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,Date_time,NO,NO2,NOX,O3,PM10,station_Barcelona (Ciutadella),station_Barcelona (Eixample),station_Barcelona (Gràcia - Sant Gervasi),station_Barcelona (Observatori Fabra),station_Barcelona (Palau Reial),station_Barcelona (Parc Vall Hebron),station_Barcelona (Poblenou),station_Barcelona (Sants)
0,2019-03-07,4.684211,30.210526,37.052632,53.263158,19.221136,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-03-07,12.850000,35.100000,54.600000,45.100000,18.900000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-03-07,9.900000,37.300000,52.050000,47.200000,17.400000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2019-03-07,12.166667,12.250000,30.416667,82.833333,8.714286,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2019-03-07,5.500000,20.850000,29.450000,51.400000,11.800000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [40]:
imputed_data.to_csv('imputed_data.csv', index=False)


In [43]:
imputed_data = pd.read_csv('imputed_data.csv')
imputed_data.head()



,Date_time,NO,NO2,NOX,O3,PM10,station_Barcelona (Ciutadella),station_Barcelona (Eixample),station_Barcelona (Gràcia - Sant Gervasi),station_Barcelona (Observatori Fabra),station_Barcelona (Palau Reial),station_Barcelona (Parc Vall Hebron),station_Barcelona (Poblenou),station_Barcelona (Sants)
0,2019-03-07,4.684211,30.210526,37.052632,53.263158,19.221136,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-03-07,12.850000,35.100000,54.600000,45.100000,18.900000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-03-07,9.900000,37.300000,52.050000,47.200000,17.400000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2019-03-07,12.166667,12.250000,30.416667,82.833333,8.714286,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2019-03-07,5.500000,20.850000,29.450000,51.400000,11.800000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [42]:

imputed_data['Date_time'] = pd.to_datetime(imputed_data['Date_time'])
imputed_data = imputed_data[imputed_data['station_Barcelona (Parc Vall Hebron)'] == 1]
imputed_data['Time_diff'] = imputed_data['Date_time'].diff().dt.days
biggest_gap = imputed_data['Time_diff'].max()
print('Biggest day difference',biggest_gap)


Biggest day difference 1.0


In [44]:
imputed_data.isna().sum()


Date_time                                    0
NO                                           0
NO2                                          0
NOX                                          0
O3                                           0
PM10                                         0
station_Barcelona (Ciutadella)               0
station_Barcelona (Eixample)                 0
station_Barcelona (Gràcia - Sant Gervasi)    0
station_Barcelona (Observatori Fabra)        0
station_Barcelona (Palau Reial)              0
station_Barcelona (Parc Vall Hebron)         0
station_Barcelona (Poblenou)                 0
station_Barcelona (Sants)                    0
dtype: int64